# Dynamics of INS errors

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import eigvals
import matplotlib.pyplot as plt
import pyins

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 12

This example demonstrates the long-term behaviour of errors of an Inertial Navigation System (INS). The errors (as long as they small enough) are described by a system of linear ODEs. When the system matrix is constant (INS resting on bench, straight flight at a constant speed) the system can be solved in a closed formed, but even in this case it is a cumbersome procedure. So pyins provides a simple routine for numerical propagation of INS errors.

## Generate Trajectory

We will use a straight and level flight as a trajectory, with a speed of 90 m/s directed eastward.
We are going to generate data for 48 hours to consider a long-term behaviour of the errors. 

In [2]:
trajectory, _ = pyins.sim.sinusoid_velocity_motion(1, 48 * 3600, [30, 30, 0], [0, 90, 0])

In [3]:
trajectory.head()

To avoid exponential divergence of altitude and vertical velocity we use a contrained error model where the corresponding errors are excluded (`with_altitude=False`).

## Eigenvalues of the error model ODEs system matrix

In our case the system matrix is constant, so we can analyze its behaviour by finding the eigenvalues.

In [4]:
error_model = pyins.error_model.InsErrorModel(with_altitude=False)

In [5]:
F, _, _ = error_model.system_matrices(trajectory.iloc[0])

In [6]:
eigvals(F)

We see that there are 3 pure imaginary complex conjugate pairs and one zero eigenvalue.
It means that the behavior of the system is oscillatory with the periods of 81.5 minutes, 87.6 minutes and 21.6 hours.
The first two are slightly perturbed so called Schuller period, which is nominally 84 minutes. 
The last one is related to Earth rotation, but with angular rate adjusted for eastward motion.
The zero eigenvalue corresponds to the cyclic property of longitude.

## Example of errors evolution without sensor errors

Here we illustrates the previous findings by computing the errors solution for some initial values.

In [7]:
def plot_errors(error):
    time = error.index / 3600
    
    plt.figure(figsize=(10, 10))
    
    plt.subplot(331)
    plt.title("north, m")
    plt.plot(time, error.north)

    plt.subplot(334)
    plt.title("east, m")
    plt.plot(time, error.east)
    
    plt.subplot(332)
    plt.title("VN, m/s")
    plt.plot(time, error.VN)
    
    plt.subplot(335)
    plt.title("VE, m/s")
    plt.plot(time, error.VE)

    plt.subplot(333)
    plt.title("roll, deg")
    plt.plot(time, error.roll)
    
    plt.subplot(336)
    plt.title("pitch, deg")
    plt.plot(time, error.pitch)
    
    plt.subplot(339)
    plt.title("heading, deg")
    plt.plot(time, error.heading)
    
    plt.suptitle("INS error vs time in hours")
    plt.tight_layout()

In [8]:
pva_error = pd.Series({
    'north': 500,
    'east': -300,
    'down': 0,
    'VN': -1,
    'VE': 2,
    'VD': 0,
    'roll': -0.02,
    'pitch': 0.01,
    'heading': 0.05
})
errors, _ = pyins.error_model.propagate_errors(trajectory, pva_error, with_altitude=False)
plot_errors(errors)

It's easy to compute responses to individual initial error components and thus compute the full transition matrix. But it will be hard and cumbersome to illustrate such result. 

A simpler way is to repeat computations with some initial errors disabled and compare the results with the full picture. By such approach the following conclusions can be easily verified:

1. An initial longitude error doesn't excite other errors and doesn't evolve itself. It means that it's impossible to distinguish between different values of longitude and the navigation can be started from any value. This component is described by the zero eigenvalue found in the system.
2. An initial latitude error doesn't influence other errors significantly.
3. Velocity, pitch and roll errors form a subsystem weakly influenced by other errors.
4. Heading error is mostly determined by initial heading, pitch and roll errors.

Note that 2, 3, 4 are true as long as $\Delta r / A$ ($\Delta r$ is the position error and $A$ is Earth radius) is significantly less than attitude errors.

## Errors due accelerometer biases

Now we investigate errors propagation when the inertial sensors containing constant biases.

We consider errors due to the East accelerometer bias (keep in mind that the attitude is constant and the heading is 90 degrees).

In [9]:
errors, _ = pyins.error_model.propagate_errors(trajectory, accel_error=[1e-2, 0, 0], with_altitude=False)
plot_errors(errors)

We see that accelerometer biases also creates oscillatory errors. The things worth noting are that east (longitude), pitch and heading errors are biased relative to zero by their oscillation amplitudes. 

Errors due to the North bias are conceptually the same, with East and North axes switched. The heading error won't have a bias.

Errors due to the vertical accelerometer bias is irrelevant for horizontal errors components.

## Errors due to gyroscope biases

First consider errors due to the East gyro bias.

In [10]:
errors, _ = pyins.error_model.propagate_errors(trajectory, gyro_error=[1e-7, 0, 0], with_altitude=False)
plot_errors(errors)

We see that errors are bounded, with east (longitude) and heading errors biased by the value of their amplitudes.

Now consider errors due to the North bias.

In [11]:
errors, _ = pyins.error_model.propagate_errors(trajectory, gyro_error=[0, -1e-7, 0], with_altitude=False)
plot_errors(errors)

This is the first case when the error in longitude growths approximately linearly to an unbounded value. 
The theoretical rate of the longitude error growth is $A \epsilon_N \cos^2 \varphi$ , where $\varphi$ is the latitude and $\epsilon_N$ is the North gyro bias. 

In our case the growth in kilometers per hour is equal to:

In [12]:
lat = np.deg2rad(trajectory.lat.iloc[0])
1e-7 * pyins.earth.A * 3.6 * np.cos(lat) ** 2

So the expected final error is 1.72 * 48 = 82.5 km, which matches well with the simulation results.

Now consider errors due to the vertical bias.

In [13]:
errors, _ = pyins.error_model.propagate_errors(trajectory, gyro_error=[0, 0, -1e-7], with_altitude=False) 
plot_errors(errors)

In this case the longitude error also growths unbounded. The rate of growth is equal to $-A \epsilon_D \sin \varphi \cos \varphi$, with $\epsilon_D$ being a gyro bias in down direction.

In [14]:
1e-7 * pyins.earth.A * 3.6 * np.sin(lat) * np.cos(lat)

And the expected final error is 0.99 * 48 = 47.5 km. The simulation value is slightly lower because of sinusoidal modulations.

## Conclusion

When the coefficients of the error equations are constant, then:

1. The horizontal INS errors contain only oscillatory modes. The two periods are 24 hours (corrected by a vehicle East velocity) and the 84 minutes (slightly perturbed depending on a latitude and a vehicle's velocity).
2. The response to constant accelerometer biases is also bounded and oscillatory. The same holds true for a constant East gyro bias.
3. In a presence of North and vertical gyro biases the longitude error growths unbounded, with the rate of growth equal to $R \varepsilon_Z$, where $\varepsilon_Z$ is projection of the gyro bias on Earth rotation axis. And this is the only unbounded error. It proves the fact that the long-term pure inertial performance of a precise INS is determined by gyroscopes accuracy.

Generally, the behaviour of INS errors can be analyzed at any level of detail and for arbitrary trajectories using pyins.